<a href="https://colab.research.google.com/github/subhendu115/Spark/blob/main/multiplt-delimeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.appName("MultiDelim").master("local[*]").getOrCreate()

In [19]:
df = spark.read.format("csv").option("sep","~|").option("header",True).load("multi-delim.dat")

In [21]:
df.show()

+---+-----------+---+
| id|       name|Age|
+---+-----------+---+
|  1|       gsfg| 23|
|  1|  ssdgdsgjs| 20|
|  1|svfgbfbggjs| 21|
|  1|     svfscv| 29|
|  1|   svcvvbjs| 23|
|  1| ssgsrhjbjs| 28|
|  1|  svfhfhbjs| 23|
|  1|  svfdgdbjs| 25|
|  1|   svfdgbjs| 22|
|  1| svdgngjbjs| 25|
+---+-----------+---+



In [38]:
df1 = spark.read.format("text").option("header",True).load("multi-delim.dat")

In [39]:
df1.show()

+------------------+
|             value|
+------------------+
|     id~|name~|Age|
|       1~|gsfg~|23|
|  1~|ssdgdsgjs~|20|
|1~|svfgbfbggjs~|21|
|     1~|svfscv~|29|
|   1~|svcvvbjs~|23|
| 1~|ssgsrhjbjs~|28|
|  1~|svfhfhbjs~|23|
|  1~|svfdgdbjs~|25|
|   1~|svfdgbjs~|22|
| 1~|svdgngjbjs~|25|
+------------------+



In [44]:
header = df1.first()[0]
header
schm = header.split('~|')
schm

['id', 'name', 'Age']

In [61]:
df_in = df1.filter(df1['value'] != header).rdd.map(lambda row:row[0].split("~|")).toDF(schm)
df_in.show()
# As the DataFarme have only One column -> hence row[0]. It can change based on number of column in dataframe/RDD

+---+-----------+---+
| id|       name|Age|
+---+-----------+---+
|  1|       gsfg| 23|
|  1|  ssdgdsgjs| 20|
|  1|svfgbfbggjs| 21|
|  1|     svfscv| 29|
|  1|   svcvvbjs| 23|
|  1| ssgsrhjbjs| 28|
|  1|  svfhfhbjs| 23|
|  1|  svfdgdbjs| 25|
|  1|   svfdgbjs| 22|
|  1| svdgngjbjs| 25|
+---+-----------+---+

